## EE4483 Project: Dogs vs Cats Classification

Import necessary libraries here

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import models
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
from torch.optim import lr_scheduler
from PIL import Image
from tqdm import tqdm
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

Use `gpu` or `cpu`

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print(torch.cuda.is_available())

Define augmentation for train and val images

In [3]:
train_transform = transforms.Compose([
                    transforms.Resize((224, 224)), 
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomRotation(15),
                    transforms.ToTensor(), 
                    transforms.Normalize([0.5]*3, [0.5]*3)
]) # train transform

test_transform = transforms.Compose([
                    transforms.Resize((224, 224)), 
                    transforms.ToTensor(), 
                    transforms.Normalize([0.5]*3, [0.5]*3)
]) # test transform

Load dataset and Print amount of images `(0-> Cat, 1-> Dog)`

In [4]:
train_dataset = ImageFolder("../datasets/train", transform=train_transform)
valid_dataset = ImageFolder("../datasets/val", transform=test_transform)
print(f"[INFO]: Number of training images: {len(train_dataset)}")
print(f"[INFO]: Number of validation images: {len(valid_dataset)}")
print(f"[INFO]: Class names: {train_dataset.classes}\n")

train_loader = DataLoader(train_dataset, batch_size = 128, shuffle=True, num_workers= 4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size = 128, num_workers= 4, pin_memory=True)

[INFO]: Number of training images: 20000
[INFO]: Number of validation images: 5000
[INFO]: Class names: ['cat', 'dog']



Visualize random batch of images in `train_loader`

In [5]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))
        break

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [6]:
show_batch(train_loader)

Load `Resnet-18` and pass to GPU

In [7]:
# load pretrain model and modify...
model = models.resnet18(pretrained=True)
# model = models.resnet18(weights=None)

# for param in model.parameters():
#     param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
    
model.to(device)

c:\Users\65913\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\65913\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Define `Train` and `Test` functions

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Learning Rate Scheduler
# scheduler = lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
save_path = '../outputs/trained_model'
learning_rate = []
training_loss = []
validation_loss = []
training_accuracy = []
valid_accuracy = []

# Train and test

def train(current_epoch, num_epoch, model):
    current_loss = 0.0
    current_corrects = 0
    losses = []
    model.train()

    loop = tqdm(enumerate(train_loader), total=len(train_loader)) # create a progress bar
    for batch_idx, (data, targets) in loop:
        data = data.to(device=device)
        targets = targets.to(device=device)
        scores = model(data)
        
        loss = criterion(scores, targets)
        optimizer.zero_grad()
        losses.append(loss)
        
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(scores, 1)
        current_loss += loss.item() * data.size(0)
        current_corrects += (preds == targets).sum().item()
        loop.set_description(f"Epoch {current_epoch+1}/{num_epoch} process: {int((batch_idx / len(train_loader)) * 100)}")
        loop.set_postfix(loss=loss.data.item())
    
    # update lr here if using stepLR
    # scheduler.step()

    epoch_accuracy = current_corrects / len(train_dataset) * 100
    epoch_losses =  current_loss / len(train_dataset)
    training_accuracy.append(epoch_accuracy)
    training_loss.append(epoch_losses)
    
    # save learning rate
    # learning_rate.append(scheduler.optimizer.param_groups[0]['lr'])
    
    # save model
    torch.save({ 
                'model_state_dict': model.state_dict(), 
                'optimizer_state_dict': optimizer.state_dict(), 
                }, os.path.join(save_path,'checpoint_epoch_'+str(current_epoch)+'.pt'))


        
# model.eval() is a kind of switch for some specific layers/parts of the model that behave differently,
# during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. 
# You need to turn off them during model evaluation, and .eval() will do it for you. In addition, 
# the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() 
# to turn off gradients computation:
        
def test(model):
    model.eval()
    current_loss = 0
    epoch_loss = 0
    correct = 0
    with torch.no_grad():
        for data, targets in valid_loader:
            data = data.to(device)
            targets = targets.to(device)
            scores = model(data)
            _, predictions = torch.max(scores, 1)
            correct += (predictions == targets).sum().item()
            loss = criterion(scores, targets)
            current_loss += loss.item() * data.size(0)
            
    epoch_loss = current_loss / len(valid_dataset)
    validation_loss.append(epoch_loss)
    valid_accuracy.append(correct / len(valid_dataset) * 100)

    print("Average Loss: ", epoch_loss, "  Accuracy: ", correct, " / ",
    len(valid_dataset), "  ", int(correct / len(valid_dataset) * 100), "%")


def train_model(num_epoch, model):
    for epoch in range(0, num_epoch):
        train(epoch, num_epoch, model)
        test(model)

In [9]:
total_epoch = 10
train_model(total_epoch, model)

Epoch 1/10 process: 99: 100%|██████████| 157/157 [00:32<00:00,  4.77it/s, loss=0.0251]


Average Loss:  0.04965275031626225   Accuracy:  4911  /  5000    98 %


Epoch 2/10 process: 99: 100%|██████████| 157/157 [00:30<00:00,  5.20it/s, loss=0.00576]


Average Loss:  0.03770399977043271   Accuracy:  4934  /  5000    98 %


Epoch 3/10 process: 99: 100%|██████████| 157/157 [00:30<00:00,  5.20it/s, loss=0.0756] 


Average Loss:  0.03546852291226387   Accuracy:  4934  /  5000    98 %


Epoch 4/10 process: 99: 100%|██████████| 157/157 [00:29<00:00,  5.27it/s, loss=0.00345]


Average Loss:  0.03265820404961705   Accuracy:  4938  /  5000    98 %


Epoch 5/10 process: 99: 100%|██████████| 157/157 [00:30<00:00,  5.21it/s, loss=0.0198] 


Average Loss:  0.030968348707258703   Accuracy:  4939  /  5000    98 %


Epoch 6/10 process: 99: 100%|██████████| 157/157 [00:29<00:00,  5.32it/s, loss=0.0874] 


Average Loss:  0.03226903419755399   Accuracy:  4938  /  5000    98 %


Epoch 7/10 process: 99: 100%|██████████| 157/157 [00:28<00:00,  5.44it/s, loss=0.0296] 


Average Loss:  0.029720890501886605   Accuracy:  4946  /  5000    98 %


Epoch 8/10 process: 99: 100%|██████████| 157/157 [00:28<00:00,  5.49it/s, loss=0.026]  


Average Loss:  0.030845038955658675   Accuracy:  4943  /  5000    98 %


Epoch 9/10 process: 99: 100%|██████████| 157/157 [00:28<00:00,  5.50it/s, loss=0.0317] 


Average Loss:  0.03051746752858162   Accuracy:  4942  /  5000    98 %


Epoch 10/10 process: 99: 100%|██████████| 157/157 [00:29<00:00,  5.25it/s, loss=0.00934]


Average Loss:  0.030717136953771113   Accuracy:  4950  /  5000    99 %


Visualize `Accuracy` and `Losses`

In [10]:
epc = []
for x in range(total_epoch):
    epc.append(x)

fig, axs = plt.subplots(1, 2, figsize=(15,10))
axs[0].set_title('Train & Val Accuracy')
axs[0].plot(epc, training_accuracy,label='train')
axs[0].plot(epc, valid_accuracy,label='val')
axs[0].legend(loc='lower right')
axs[1].set_title('Train & Val Loss')
axs[1].plot(epc, training_loss,label='train')
axs[1].plot(epc, validation_loss,label='val')
axs[1].legend(loc='upper right')

Load the trained model

In [ ]:
print("----> Loading checkpoint")
checkpoint = torch.load("../outputs/trained_model/checpoint_epoch_9.pt") # Try to load last checkpoint
model.load_state_dict(checkpoint["model_state_dict"]) 
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

Load test images into `DataLoader` and input to model

In [ ]:
# Custom function to load test images
class CatDogDataset(Dataset):
    
    def __init__(self, imgs, transform = None):
        
        super().__init__()
        self.imgs = imgs
        self.transform = transform
        
    def __getitem__(self, idx):
        
        image_name = self.imgs[idx]
        img = Image.open(os.path.join(TEST_DIR, image_name))
        
        ### Apply Transforms on image
        img = self.transform(img)

        return img
            
        
    def __len__(self):
        return len(self.imgs)

In [ ]:
TEST_DIR = '../datasets/test'
id = []
test_imgs = os.listdir(TEST_DIR)
test_imgs = sorted(test_imgs, key=lambda x: int(x.split(".")[0]))
for image in test_imgs:
    id.append(image.split(".")[0])
# print(id)



test_transform = transforms.Compose([
                         transforms.Resize((224, 224)), 
                         transforms.ToTensor(), 
                         transforms.Normalize([0.5]*3, [0.5]*3)
                     ])

# Check the test set
dataset = CatDogDataset(test_imgs, 
                     transform=test_transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle = False)

predictions = []
with torch.no_grad():
    model.eval()
    for data in dataloader:
        data = data.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        predictions.append(predicted[0].item()) # store all predictions in array

Index to Class Conversion (If necessary)

In [ ]:
# Convert class idx to string (0->cat, 1->dog)
string_label = []
for label in predictions:
    if label == 0:
        string_label.append('cat')
    elif label == 1:
        string_label.append("dog")
    else:
        string_label.append("null")

Create submission csv

In [ ]:
d = {'id':id, 'label':predictions}
output_df = pd.DataFrame(data=d)
output_df.to_csv('../outputs/submission.csv',index=False)